In [187]:
import datetime
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from IPython.display import display
pd.options.display.max_columns = None


from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0']

In [70]:
num_rows = None
train_df = pd.read_csv('/Users/jincui/Downloads/ELO/train.csv', index_col=['card_id'], nrows=num_rows)
test_df = pd.read_csv('/Users/jincui/Downloads/ELO/test.csv', index_col=['card_id'], nrows=num_rows)

print("Train samples: {}, test samples: {}".format(len(train_df), len(test_df)))

# outlier
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1

# set target as nan
test_df['target'] = np.nan

# merge
df = train_df.append(test_df)
df


Train samples: 201917, test samples: 123623


,feature_1,feature_2,feature_3,first_active_month,outliers,target
card_id,,,,,,
C_ID_92a2005557,5,2,1,2017-06,0.0,-0.820283
C_ID_3d0044924f,4,1,0,2017-01,0.0,0.392913
C_ID_d639edf6cd,2,2,0,2016-08,0.0,0.688056
C_ID_186d6a6901,4,3,0,2017-09,0.0,0.142495
C_ID_cdbd2c0db2,1,3,0,2017-11,0.0,-0.159749
C_ID_0894217f2f,4,2,0,2016-09,0.0,0.871585
C_ID_7e63323c00,3,2,1,2016-12,0.0,0.230129
C_ID_dfa21fc124,3,2,1,2017-09,0.0,2.135850
C_ID_fe0fdac8ea,2,1,0,2017-08,0.0,-0.065406


In [71]:
df = df.rename(columns={"first_active_month": "first_active_time"})

df['first_active_time'] = pd.to_datetime(df['first_active_time'])

In [72]:
# there is one where card_id == C_ID_c27b4f80f7, fist_active_month infomration is missing

df['first_active_year'] = df['first_active_time'].dt.year
df['first_active_month'] = df['first_active_time'].dt.month
df

,feature_1,feature_2,feature_3,first_active_time,outliers,target,first_active_year,first_active_month
card_id,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06-01,0.0,-0.820283,2017.0,6.0
C_ID_3d0044924f,4,1,0,2017-01-01,0.0,0.392913,2017.0,1.0
C_ID_d639edf6cd,2,2,0,2016-08-01,0.0,0.688056,2016.0,8.0
C_ID_186d6a6901,4,3,0,2017-09-01,0.0,0.142495,2017.0,9.0
C_ID_cdbd2c0db2,1,3,0,2017-11-01,0.0,-0.159749,2017.0,11.0
C_ID_0894217f2f,4,2,0,2016-09-01,0.0,0.871585,2016.0,9.0
C_ID_7e63323c00,3,2,1,2016-12-01,0.0,0.230129,2016.0,12.0
C_ID_dfa21fc124,3,2,1,2017-09-01,0.0,2.135850,2017.0,9.0
C_ID_fe0fdac8ea,2,1,0,2017-08-01,0.0,-0.065406,2017.0,8.0


In [ ]:
## features are type of card, e.g.
## they have certai range of first_active_time

In [74]:
df.groupby('feature_1').min()

,feature_2,feature_3,first_active_time,outliers,target,first_active_year,first_active_month
feature_1,,,,,,,
1,1,0,2015-01-01,0.0,-33.219281,2015.0,1.0
2,1,0,2014-11-01,0.0,-33.219281,2014.0,1.0
3,1,1,2011-11-01,0.0,-33.219281,2011.0,1.0
4,1,0,2015-01-01,0.0,-33.219281,2015.0,1.0
5,1,1,2013-11-01,0.0,-33.219281,2013.0,1.0


In [76]:
df.groupby('feature_1').max()

,feature_2,feature_3,first_active_time,outliers,target,first_active_year,first_active_month
feature_1,,,,,,,
1,3,0,2018-02-01,1.0,15.011358,2018.0,12.0
2,3,0,2018-01-01,1.0,13.592223,2018.0,12.0
3,3,1,2018-01-01,1.0,15.606405,2018.0,12.0
4,3,0,2018-01-01,1.0,9.665204,2018.0,12.0
5,2,1,2018-01-01,1.0,17.965068,2018.0,12.0


In [49]:
df.groupby('feature_2').min()

,feature_1,feature_3,first_active_month,outliers,target,first_active_year
feature_2,,,,,,
1,1,0,1.0,0.0,-33.219281,2011.0
2,1,0,1.0,0.0,-33.219281,2013.0
3,1,0,1.0,0.0,-33.219281,2011.0


In [77]:
df.groupby('feature_2').max()

,feature_1,feature_3,first_active_time,outliers,target,first_active_year,first_active_month
feature_2,,,,,,,
1,5,1,2018-01-01,1.0,17.965068,2018.0,12.0
2,5,1,2018-02-01,1.0,15.606405,2018.0,12.0
3,4,1,2018-01-01,1.0,13.350895,2018.0,12.0


In [75]:
df.groupby('feature_3').min()

,feature_1,feature_2,first_active_time,outliers,target,first_active_year,first_active_month
feature_3,,,,,,,
0,1,1,2014-11-01,0.0,-33.219281,2014.0,1.0
1,3,1,2011-11-01,0.0,-33.219281,2011.0,1.0


In [ ]:
## if feature_1 == 3 or 5: feature_3 == 1
## else feature_3 == 0!
##
### feature_1 == 4 card won't have a very high target

In [79]:
df.groupby(['feature_1', 'feature_2']).min()

feature_3 first_active_time  outliers     target  \
feature_1 feature_2                                                     
1         1                  0        2016-06-01       0.0 -33.219281   
          2                  0        2015-01-01       0.0 -33.219281   
          3                  0        2016-04-01       0.0 -33.219281   
2         1                  0        2014-11-01       0.0 -33.219281   
          2                  0        2015-01-01       0.0 -33.219281   
          3                  0        2016-04-01       0.0 -33.219281   
3         1                  1        2011-11-01       0.0 -33.219281   
          2                  1        2013-04-01       0.0 -33.219281   
          3                  1        2011-11-01       0.0 -33.219281   
4         1                  0        2016-06-01       0.0 -33.219281   
          2                  0        2015-01-01       0.0 -33.219281   
          3                  0        2016-04-01       0.0 -33.219281   
5         1                  1        2013-11-01       0.0 -33.219281   
          2                  1        2016-05-01       0.0 -33.219281   

                     first_active_year  first_active_month  
feature_1 feature_2                                         
1         1                     2016.0                 1.0  
          2                     2015.0                 1.0  
          3                     2016.0                 1.0  
2         1                     2014.0                 1.0  
          2                     2015.0                 1.0  
          3                     2016.0                 1.0  
3         1                     2011.0                 1.0  
          2                     2013.0                 1.0  
          3                     2011.0                 1.0  
4         1                     2016.0                 1.0  
          2                     2015.0                 1.0  
          3                     2016.0                 1.0  
5         1                     2013.0                 1.0  
          2                     2016.0                 1.0

In [80]:
df.groupby(['feature_1', 'feature_2']).max()

feature_3 first_active_time  outliers     target  \
feature_1 feature_2                                                     
1         1                  0        2018-01-01       1.0  15.011358   
          2                  0        2018-02-01       1.0  11.553597   
          3                  0        2017-12-01       1.0  10.139105   
2         1                  0        2018-01-01       1.0  13.592223   
          2                  0        2018-01-01       1.0  10.474853   
          3                  0        2018-01-01       1.0  13.350895   
3         1                  1        2018-01-01       1.0  12.446308   
          2                  1        2018-01-01       1.0  15.606405   
          3                  1        2018-01-01       1.0  11.291580   
4         1                  0        2018-01-01       1.0   9.665204   
          2                  0        2018-01-01       1.0   9.365076   
          3                  0        2018-01-01       1.0   8.867564   
5         1                  1        2018-01-01       1.0  17.965068   
          2                  1        2018-01-01       1.0  15.294621   

                     first_active_year  first_active_month  
feature_1 feature_2                                         
1         1                     2018.0                12.0  
          2                     2018.0                12.0  
          3                     2017.0                12.0  
2         1                     2018.0                12.0  
          2                     2018.0                12.0  
          3                     2018.0                12.0  
3         1                     2018.0                12.0  
          2                     2018.0                12.0  
          3                     2018.0                12.0  
4         1                     2018.0                12.0  
          2                     2018.0                12.0  
          3                     2018.0                12.0  
5         1                     2018.0                12.0  
          2                     2018.0                12.0

In [90]:
df.groupby(['feature_2', 'feature_1']).min()

feature_3 first_active_time  outliers     target  \
feature_2 feature_1                                                     
1         1                  0        2016-06-01       0.0 -33.219281   
          2                  0        2014-11-01       0.0 -33.219281   
          3                  1        2011-11-01       0.0 -33.219281   
          4                  0        2016-06-01       0.0 -33.219281   
          5                  1        2013-11-01       0.0 -33.219281   
2         1                  0        2015-01-01       0.0 -33.219281   
          2                  0        2015-01-01       0.0 -33.219281   
          3                  1        2013-04-01       0.0 -33.219281   
          4                  0        2015-01-01       0.0 -33.219281   
          5                  1        2016-05-01       0.0 -33.219281   
3         1                  0        2016-04-01       0.0 -33.219281   
          2                  0        2016-04-01       0.0 -33.219281   
          3                  1        2011-11-01       0.0 -33.219281   
          4                  0        2016-04-01       0.0 -33.219281   

                     first_active_year  first_active_month  
feature_2 feature_1                                         
1         1                     2016.0                 1.0  
          2                     2014.0                 1.0  
          3                     2011.0                 1.0  
          4                     2016.0                 1.0  
          5                     2013.0                 1.0  
2         1                     2015.0                 1.0  
          2                     2015.0                 1.0  
          3                     2013.0                 1.0  
          4                     2015.0                 1.0  
          5                     2016.0                 1.0  
3         1                     2016.0                 1.0  
          2                     2016.0                 1.0  
          3                     2011.0                 1.0  
          4                     2016.0                 1.0

In [89]:
df.groupby(['feature_2', 'feature_1']).max()

feature_3 first_active_time  outliers     target  \
feature_2 feature_1                                                     
1         1                  0        2018-01-01       1.0  15.011358   
          2                  0        2018-01-01       1.0  13.592223   
          3                  1        2018-01-01       1.0  12.446308   
          4                  0        2018-01-01       1.0   9.665204   
          5                  1        2018-01-01       1.0  17.965068   
2         1                  0        2018-02-01       1.0  11.553597   
          2                  0        2018-01-01       1.0  10.474853   
          3                  1        2018-01-01       1.0  15.606405   
          4                  0        2018-01-01       1.0   9.365076   
          5                  1        2018-01-01       1.0  15.294621   
3         1                  0        2017-12-01       1.0  10.139105   
          2                  0        2018-01-01       1.0  13.350895   
          3                  1        2018-01-01       1.0  11.291580   
          4                  0        2018-01-01       1.0   8.867564   

                     first_active_year  first_active_month  
feature_2 feature_1                                         
1         1                     2018.0                12.0  
          2                     2018.0                12.0  
          3                     2018.0                12.0  
          4                     2018.0                12.0  
          5                     2018.0                12.0  
2         1                     2018.0                12.0  
          2                     2018.0                12.0  
          3                     2018.0                12.0  
          4                     2018.0                12.0  
          5                     2018.0                12.0  
3         1                     2017.0                12.0  
          2                     2018.0                12.0  
          3                     2018.0                12.0  
          4                     2018.0                12.0

In [241]:
merchants = pd.read_csv('/Users/jincui/Downloads/ELO/merchants.csv', nrows=num_rows)
#merchants

In [102]:
merchants.groupby('category_4').min()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,city_id,state_id,category_2
category_4,,,,,,,,,,,,,,,,,,,,
N,1,-1,-1,-0.057471,-0.057471,N,A,A,-82.13,0.333495,1,-82.13,0.167045,1,-82.13,0.098330,1,-1,-1,1.0
Y,4,2,1,-0.057471,-0.057471,N,A,A,0.33,0.336570,1,0.16,0.174775,1,0.09,0.102951,1,-1,-1,1.0


In [101]:
merchants.groupby('category_4').max()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,city_id,state_id,category_2
category_4,,,,,,,,,,,,,,,,,,,,
N,112585,891,41,183.735111,182.079322,N,E,E,624741.00,inf,3,1513959.00,inf,6,2567408.00,inf,12,347,24,5.0
Y,112586,891,41,172.719653,170.736672,Y,E,E,851844.64,inf,3,770443.34,inf,6,689561.85,inf,12,320,24,5.0


In [185]:
hist_df = pd.read_csv('/Users/jincui/Downloads/ELO/historical_transactions.csv', nrows=num_rows)
new_merchant_df = pd.read_csv('/Users/jincui/Downloads/ELO/new_merchant_transactions.csv', nrows=num_rows)

In [110]:
merchants[['merchant_id','merchant_group_id', 'merchant_category_id',
           'subsector_id', 'category_4', 'city_id', 'state_id', 
           'category_2', 'category_1']][merchants['merchant_id']=='M_ID_e020e9b302']

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,category_4,city_id,state_id,category_2,category_1
334495,M_ID_e020e9b302,35,80,37,Y,88,16,1.0,N


In [111]:
hist_df.shape

(29112361, 14)

In [240]:
MERCHANT_FEATURES_EXCLUDE = ['merchant_category_id', 'category_2', 'category_1', 'category_4', 'city_id', 'state_id', 'subsector_id']

merchants_simp = merchants.drop(MERCHANT_FEATURES_EXCLUDE, axis=1)
#merchants_simp

In [242]:
merchants['numerical_ratio'] = merchants['numerical_1']/merchants['numerical_2']
#merchants_simp


In [243]:
dummies_columns = ['most_recent_sales_range','most_recent_purchases_range', 'active_months_lag3', 'active_months_lag6', 'active_months_lag12', 'category_1', 'category_2', 'category_4']


merchants = pd.get_dummies(merchants,
                            columns = dummies_columns,
                            prefix = dummies_columns,
                            dummy_na=True)
merchants


,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,city_id,state_id,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_sales_range_nan,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,most_recent_purchases_range_nan,active_months_lag3_1.0,active_months_lag3_2.0,active_months_lag3_3.0,active_months_lag3_nan,active_months_lag6_1.0,active_months_lag6_2.0,active_months_lag6_3.0,active_months_lag6_4.0,active_months_lag6_5.0,active_months_lag6_6.0,active_months_lag6_nan,active_months_lag12_1.0,active_months_lag12_2.0,active_months_lag12_3.0,active_months_lag12_4.0,active_months_lag12_5.0,active_months_lag12_6.0,active_months_lag12_7.0,active_months_lag12_8.0,active_months_lag12_9.0,active_months_lag12_10.0,active_months_lag12_11.0,active_months_lag12_12.0,active_months_lag12_nan,category_1_N,category_1_Y,category_1_nan,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_4_N,category_4_Y,category_4_nan
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,-0.40,9.666667,-2.25,18.666667,-2.32,13.916667,242,9,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,-0.72,1.750000,-0.74,1.291667,-0.57,1.687500,22,16,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,-82.13,260.000000,-82.13,260.000000,-82.13,260.000000,-1,5,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,NaN,1.666667,NaN,4.666667,NaN,3.833333,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,NaN,0.500000,NaN,0.361111,NaN,0.347222,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
5,M_ID_a0915f62b5,20201,87,27,-0.057471,-0.057471,NaN,1.000000,NaN,3.666667,NaN,3.833333,160,21,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
6,M_ID_bfd41933db,33861,792,9,-0.057471,-0.057471,NaN,2.833333,NaN,4.833333,NaN,6.333333,60,16,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
7,M_ID_d8ff08219e,16430,529,20,-0.057471,-0.057471,NaN,1.000000,NaN,1.666667,NaN,1.500000,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0
8,M_ID_c5b389236d,37179,813,29,-0.057471,-0.057471,NaN,115.000000,NaN,189.916667,NaN,197.000000,248,15,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
9,M_ID_d2162ed113,112122,81,29,-0.057471,-0.057471,NaN,1.000000,NaN,1.000000,NaN,1.000000,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0


In [245]:
# some merchant_id is duplicated
merchants.merchant_id.value_counts()

M_ID_c2b9ac2ea4    4
M_ID_dbbf07ebf0    4
M_ID_d123532c72    4
M_ID_ef233cff26    4
M_ID_6464db3b45    4
M_ID_992a180b15    4
M_ID_42697d5d44    4
M_ID_1802942aaf    4
M_ID_bd49e37dda    4
M_ID_30340088f2    4
M_ID_ebbdb42da6    4
M_ID_1ceca881f0    2
M_ID_28c2aace87    2
M_ID_0b8ed0c2b0    2
M_ID_00a6ca8a8a    2
M_ID_d2b5d4418d    2
M_ID_49daf8a838    2
M_ID_fea38c640b    2
M_ID_8951e22556    2
M_ID_7483dde87d    2
M_ID_32d73b8ecf    2
M_ID_0a00fa9e8a    2
M_ID_0039220eb3    2
M_ID_654509ddbc    2
M_ID_ae9fe1605a    2
M_ID_0c4018d3a0    2
M_ID_26d4fadb60    2
M_ID_57e97f6546    2
M_ID_b794b9d9e8    2
M_ID_645a6af169    2
                  ..
M_ID_1b3907a3d8    1
M_ID_f100cacd9f    1
M_ID_8607c65471    1
M_ID_02ef1090db    1
M_ID_bc8986ae48    1
M_ID_5d1d88fd09    1
M_ID_32ef1124e0    1
M_ID_eaceb9f3da    1
M_ID_e5b25d1000    1
M_ID_0d390d80c1    1
M_ID_bed3962027    1
M_ID_480c574653    1
M_ID_e7105e4a6d    1
M_ID_e28710256a    1
M_ID_6860fc2f6a    1
M_ID_cafec31641    1
M_ID_5486480e

In [217]:
merchants[merchants['merchant_id']=='M_ID_c2b9ac2ea4']

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,category_4,city_id,state_id,category_2,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,active_months_lag3_1,active_months_lag3_2,active_months_lag3_3,active_months_lag6_1,active_months_lag6_2,active_months_lag6_3,active_months_lag6_4,active_months_lag6_5,active_months_lag6_6,active_months_lag12_1,active_months_lag12_2,active_months_lag12_3,active_months_lag12_4,active_months_lag12_5,active_months_lag12_6,active_months_lag12_7,active_months_lag12_8,active_months_lag12_9,active_months_lag12_10,active_months_lag12_11,active_months_lag12_12
210653,M_ID_c2b9ac2ea4,69907,554,25,-0.057471,-0.057471,Y,1.11,0.702381,1.11,0.702381,1.11,0.702381,Y,-1,-1,NaN,1.0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
210654,M_ID_c2b9ac2ea4,35,554,25,-0.057471,-0.057471,Y,1.11,0.702381,1.11,0.702381,1.11,0.702381,Y,-1,-1,NaN,1.0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
210655,M_ID_c2b9ac2ea4,35,554,25,-0.057471,-0.057471,Y,2.13,1.806607,2.02,1.572673,1.77,1.491291,Y,-1,-1,NaN,1.0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
210656,M_ID_c2b9ac2ea4,69907,554,25,-0.057471,-0.057471,Y,2.13,1.806607,2.02,1.572673,1.77,1.491291,Y,-1,-1,NaN,1.0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [218]:
# this merchant_id has different merchant_category_id, city_id and state_id!!!
## but only the first one is found in hist_df, and none is found in new_merchant_df
merchants[merchants['merchant_id']=='M_ID_645a6af169']


,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,category_4,city_id,state_id,category_2,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,active_months_lag3_1,active_months_lag3_2,active_months_lag3_3,active_months_lag6_1,active_months_lag6_2,active_months_lag6_3,active_months_lag6_4,active_months_lag6_5,active_months_lag6_6,active_months_lag12_1,active_months_lag12_2,active_months_lag12_3,active_months_lag12_4,active_months_lag12_5,active_months_lag12_6,active_months_lag12_7,active_months_lag12_8,active_months_lag12_9,active_months_lag12_10,active_months_lag12_11,active_months_lag12_12
47803,M_ID_645a6af169,35926,68,27,-0.057471,-0.057471,N,1.69,0.791667,2.62,1.812500,2.30,1.458333,N,94,19,3.0,1.0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
47804,M_ID_645a6af169,19140,87,27,-0.057471,-0.057471,N,0.89,1.111111,1.54,1.555556,1.55,2.000000,N,29,15,1.0,1.0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [184]:
hist_df[hist_df['merchant_id']=='M_ID_645a6af169']

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
3356832,Y,C_ID_98f5accdf1,94,N,2,C,87,M_ID_645a6af169,-10,0.749733,2017-04-27 20:42:01,3.0,19,27
9745698,Y,C_ID_53ffaf6b3b,94,N,0,A,87,M_ID_645a6af169,-7,-0.605659,2017-07-03 17:49:02,3.0,19,27
9933669,Y,C_ID_937c6664d1,94,N,0,A,87,M_ID_645a6af169,-7,-0.614374,2017-07-26 17:31:14,3.0,19,27
13594504,N,C_ID_979f05c4b4,94,N,3,C,87,M_ID_645a6af169,-8,-0.357721,2017-06-21 18:42:51,3.0,19,27
17290873,Y,C_ID_7ee81ee0c5,29,N,0,A,87,M_ID_645a6af169,-9,-0.621887,2017-05-18 19:20:52,1.0,15,27
23495271,Y,C_ID_7989f35725,94,N,-1,NaN,68,M_ID_645a6af169,-1,-0.163879,2018-01-06 16:15:09,3.0,19,27
27031946,Y,C_ID_37ef62ae47,94,N,-1,NaN,68,M_ID_645a6af169,-2,-0.117297,2017-12-20 19:59:36,3.0,19,27


In [186]:
new_merchant_df[new_merchant_df['merchant_id']=='M_ID_645a6af169']

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id


In [219]:
merchants[['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12']].corr()

,avg_sales_lag3,avg_sales_lag6,avg_sales_lag12
avg_sales_lag3,1.000000,0.818022,0.634283
avg_sales_lag6,0.818022,1.000000,0.958423
avg_sales_lag12,0.634283,0.958423,1.000000


In [220]:
merchants[['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']].corr()

,avg_purchases_lag3,avg_purchases_lag6,avg_purchases_lag12
avg_purchases_lag3,1.000000,0.996669,0.989830
avg_purchases_lag6,0.996669,1.000000,0.997549
avg_purchases_lag12,0.989830,0.997549,1.000000


In [250]:
merchants.isna().sum()

merchant_id                         0
merchant_group_id                   0
merchant_category_id                0
subsector_id                        0
numerical_1                         0
numerical_2                         0
avg_sales_lag3                     13
avg_purchases_lag3                  0
avg_sales_lag6                     13
avg_purchases_lag6                  0
avg_sales_lag12                    13
avg_purchases_lag12                 0
city_id                             0
state_id                            0
numerical_ratio                     0
most_recent_sales_range_A           0
most_recent_sales_range_B           0
most_recent_sales_range_C           0
most_recent_sales_range_D           0
most_recent_sales_range_E           0
most_recent_sales_range_nan         0
most_recent_purchases_range_A       0
most_recent_purchases_range_B       0
most_recent_purchases_range_C       0
most_recent_purchases_range_D       0
most_recent_purchases_range_E       0
most_recent_

In [251]:
merchants[merchants['avg_sales_lag3'].isnull()]

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,city_id,state_id,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_sales_range_nan,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,most_recent_purchases_range_nan,active_months_lag3_1.0,active_months_lag3_2.0,active_months_lag3_3.0,active_months_lag3_nan,active_months_lag6_1.0,active_months_lag6_2.0,active_months_lag6_3.0,active_months_lag6_4.0,active_months_lag6_5.0,active_months_lag6_6.0,active_months_lag6_nan,active_months_lag12_1.0,active_months_lag12_2.0,active_months_lag12_3.0,active_months_lag12_4.0,active_months_lag12_5.0,active_months_lag12_6.0,active_months_lag12_7.0,active_months_lag12_8.0,active_months_lag12_9.0,active_months_lag12_10.0,active_months_lag12_11.0,active_months_lag12_12.0,active_months_lag12_nan,category_1_N,category_1_Y,category_1_nan,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_4_N,category_4_Y,category_4_nan
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,NaN,1.666667,NaN,4.666667,NaN,3.833333,-1,-1,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,NaN,0.500000,NaN,0.361111,NaN,0.347222,-1,-1,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
5,M_ID_a0915f62b5,20201,87,27,-0.057471,-0.057471,NaN,1.000000,NaN,3.666667,NaN,3.833333,160,21,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
6,M_ID_bfd41933db,33861,792,9,-0.057471,-0.057471,NaN,2.833333,NaN,4.833333,NaN,6.333333,60,16,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
7,M_ID_d8ff08219e,16430,529,20,-0.057471,-0.057471,NaN,1.000000,NaN,1.666667,NaN,1.500000,-1,-1,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0
8,M_ID_c5b389236d,37179,813,29,-0.057471,-0.057471,NaN,115.000000,NaN,189.916667,NaN,197.000000,248,15,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
9,M_ID_d2162ed113,112122,81,29,-0.057471,-0.057471,NaN,1.000000,NaN,1.000000,NaN,1.000000,-1,-1,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
10,M_ID_492cfa500c,13462,369,27,-0.057471,-0.057471,NaN,inf,NaN,inf,NaN,inf,-1,9,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
11,M_ID_73487fed26,17123,427,27,-0.057471,-0.057471,NaN,inf,NaN,inf,NaN,inf,-1,-1,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
12,M_ID_7149162139,2118,63,27,-0.057471,-0.057471,NaN,inf,NaN,inf,NaN,inf,88,16,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0


In [254]:
# these 3 merchants contain inf avg_purchase_lag
# it indicates on transaction in last month!
merchants[~np.isfinite(merchants['avg_purchases_lag3'])]

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,city_id,state_id,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_sales_range_nan,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,most_recent_purchases_range_nan,active_months_lag3_1.0,active_months_lag3_2.0,active_months_lag3_3.0,active_months_lag3_nan,active_months_lag6_1.0,active_months_lag6_2.0,active_months_lag6_3.0,active_months_lag6_4.0,active_months_lag6_5.0,active_months_lag6_6.0,active_months_lag6_nan,active_months_lag12_1.0,active_months_lag12_2.0,active_months_lag12_3.0,active_months_lag12_4.0,active_months_lag12_5.0,active_months_lag12_6.0,active_months_lag12_7.0,active_months_lag12_8.0,active_months_lag12_9.0,active_months_lag12_10.0,active_months_lag12_11.0,active_months_lag12_12.0,active_months_lag12_nan,category_1_N,category_1_Y,category_1_nan,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_4_N,category_4_Y,category_4_nan
10,M_ID_492cfa500c,13462,369,27,-0.057471,-0.057471,NaN,inf,NaN,inf,NaN,inf,-1,9,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
11,M_ID_73487fed26,17123,427,27,-0.057471,-0.057471,NaN,inf,NaN,inf,NaN,inf,-1,-1,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
12,M_ID_7149162139,2118,63,27,-0.057471,-0.057471,NaN,inf,NaN,inf,NaN,inf,88,16,1.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0


In [264]:
merchants[['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']][np.isfinite(merchants['avg_purchases_lag3'])].describe()


,avg_purchases_lag3,avg_purchases_lag6,avg_purchases_lag12
count,334693.000000,334693.000000,334693.000000
mean,1.590762,1.887568,2.079195
std,107.187059,97.862790,88.442384
min,0.333495,0.167045,0.098330
25%,0.923650,0.902245,0.898333
50%,1.016667,1.026961,1.043360
75%,1.146520,1.215556,1.266461
max,61851.333333,56077.500000,50215.555556


In [266]:
merchants['has_last_month_transaction'] = 1
merchants['has_last_month_transaction'][~np.isfinite(merchants['avg_purchases_lag3'])] = 0


In [267]:
merchants['has_last3_months_revenue'] = 1
merchants['has_last3_months_revenue'][merchants['avg_sales_lag3'].isnull()] = 0

In [269]:
merchants['avg_purchases_lag3'].replace(np.inf, 61851.333333, inplace=True)
merchants['avg_purchases_lag6'].replace(np.inf, 56077.500000, inplace=True)
merchants['avg_purchases_lag12'].replace(np.inf, 50215.555556, inplace=True)


In [272]:
merchants[['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12']].replace(np.nan, 0, inplace=True)

In [278]:
merchants[['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12']] = merchants[['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12']].fillna(0)

In [279]:
merchants

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,city_id,state_id,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_sales_range_nan,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,most_recent_purchases_range_nan,active_months_lag3_1.0,active_months_lag3_2.0,active_months_lag3_3.0,active_months_lag3_nan,active_months_lag6_1.0,active_months_lag6_2.0,active_months_lag6_3.0,active_months_lag6_4.0,active_months_lag6_5.0,active_months_lag6_6.0,active_months_lag6_nan,active_months_lag12_1.0,active_months_lag12_2.0,active_months_lag12_3.0,active_months_lag12_4.0,active_months_lag12_5.0,active_months_lag12_6.0,active_months_lag12_7.0,active_months_lag12_8.0,active_months_lag12_9.0,active_months_lag12_10.0,active_months_lag12_11.0,active_months_lag12_12.0,active_months_lag12_nan,category_1_N,category_1_Y,category_1_nan,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_4_N,category_4_Y,category_4_nan,has_last_month_transaction,has_last3_months_revenue
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,-0.40,9.666667,-2.25,18.666667,-2.32,13.916667,242,9,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,-0.72,1.750000,-0.74,1.291667,-0.57,1.687500,22,16,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,-82.13,260.000000,-82.13,260.000000,-82.13,260.000000,-1,5,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,0.00,1.666667,0.00,4.666667,0.00,3.833333,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,0.00,0.500000,0.00,0.361111,0.00,0.347222,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
5,M_ID_a0915f62b5,20201,87,27,-0.057471,-0.057471,0.00,1.000000,0.00,3.666667,0.00,3.833333,160,21,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0
6,M_ID_bfd41933db,33861,792,9,-0.057471,-0.057471,0.00,2.833333,0.00,4.833333,0.00,6.333333,60,16,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
7,M_ID_d8ff08219e,16430,529,20,-0.057471,-0.057471,0.00,1.000000,0.00,1.666667,0.00,1.500000,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
8,M_ID_c5b389236d,37179,813,29,-0.057471,-0.057471,0.00,115.000000,0.00,189.916667,0.00,197.000000,248,15,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
9,M_ID_d2162ed113,112122,81,29,-0.057471,-0.057471,0.00,1.000000,0.00,1.000000,0.00,1.000000,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0


In [328]:
z = hist_df.groupby('card_id')['purchase_date'].max()
q = hist_df.groupby('card_id')['purchase_date'].min()
z = pd.DataFrame(z)
z.columns = ['Max']
z.reset_index(level=0, inplace=True)
q = pd.DataFrame(q)
q.columns = ['Min']
q.reset_index(level=0, inplace=True)

now = datetime.datetime.now()
curr_date = now.strftime("%m-%d-%Y, %H:%M:%S")
curr_date = pd.to_datetime(curr_date)

In [334]:
rec = pd.merge(z, q, how='left', on='card_id')
rec['Min'] = pd.to_datetime(rec['Min'])
rec['Max'] = pd.to_datetime(rec['Max'])

In [337]:
rec['Recency'] = (curr_date - rec['Max']).astype('timedelta64[D]')

# Recency
      Recency = timelapse of the last transaction
      Time = timelapse of the first transaction

In [343]:
rec['Time'] = (rec['Max'] - rec['Min']).astype('timedelta64[D]')
rec = rec[['card_id', 'Recency', 'Time']]
rec

,card_id,Recency,Time
0,C_ID_00007093c1,337.0,377.0
1,C_ID_0001238066,336.0,151.0
2,C_ID_0001506ef0,346.0,398.0
3,C_ID_0001793786,455.0,283.0
4,C_ID_000183fdda,338.0,202.0
5,C_ID_00024e244b,357.0,399.0
6,C_ID_0002709b5a,344.0,402.0
7,C_ID_00027503e2,375.0,237.0
8,C_ID_000298032a,370.0,345.0
9,C_ID_0002ba3c2e,457.0,295.0


In [340]:
# Frequency is the number of swipes

freq = hist_df.groupby('card_id').size().reset_index()
freq.columns = ['card_id', 'Frequency']
freq.head()

,card_id,Frequency
0,C_ID_00007093c1,149
1,C_ID_0001238066,123
2,C_ID_0001506ef0,66
3,C_ID_0001793786,216
4,C_ID_000183fdda,144


In [341]:
# Monitary is sum of purchase amount

mon = hist_df.groupby('card_id')['purchase_amount'].sum().reset_index()
mon.columns = ['card_id', 'Monitary']
mon.head()

,card_id,Monitary
0,C_ID_00007093c1,-76.845041
1,C_ID_0001238066,-72.447201
2,C_ID_0001506ef0,-34.601879
3,C_ID_0001793786,-36.786013
4,C_ID_000183fdda,-68.837938


In [344]:
final = pd.merge(freq, mon, how='left', on='card_id')
final = pd.merge(final, rec, how='left', on='card_id')
final.head()

,card_id,Frequency,Monitary,Recency,Time
0,C_ID_00007093c1,149,-76.845041,337.0,377.0
1,C_ID_0001238066,123,-72.447201,336.0,151.0
2,C_ID_0001506ef0,66,-34.601879,346.0,398.0
3,C_ID_0001793786,216,-36.786013,455.0,283.0
4,C_ID_000183fdda,144,-68.837938,338.0,202.0


In [345]:
final['historic_CLV'] = final['Frequency'] * final['Monitary']
final['AOV'] = final['Monitary']/final['Frequency']
final['Predictive_CLV'] = final['Time']*final['AOV']*final['Monitary']*final['Recency'] 
final.head()

,card_id,Frequency,Monitary,Recency,Time,historic_CLV,AOV,Predictive_CLV
0,C_ID_00007093c1,149,-76.845041,337.0,377.0,-11449.911042,-0.515739,5.035199e+06
1,C_ID_0001238066,123,-72.447201,336.0,151.0,-8911.005737,-0.589002,2.164982e+06
2,C_ID_0001506ef0,66,-34.601879,346.0,398.0,-2283.724033,-0.524271,2.498128e+06
3,C_ID_0001793786,216,-36.786013,455.0,283.0,-7945.778765,-0.170306,8.066953e+05
4,C_ID_000183fdda,144,-68.837938,338.0,202.0,-9912.663098,-0.478041,2.246784e+06


In [395]:
final.columns

Index(['card_id', 'Frequency', 'Monitary', 'Recency', 'Time', 'historic_CLV',
       'AOV', 'Predictive_CLV'],
      dtype='object')

In [357]:
z = new_merchant_df.groupby('card_id')['purchase_date'].max()
q = new_merchant_df.groupby('card_id')['purchase_date'].min()
z = pd.DataFrame(z)
z.columns = ['Max']
z.reset_index(level=0, inplace=True)
q = pd.DataFrame(q)
q.columns = ['Min']
q.reset_index(level=0, inplace=True)

In [359]:
rec = pd.merge(z, q, how='left', on='card_id')
rec['Min'] = pd.to_datetime(rec['Min'])
rec['Max'] = pd.to_datetime(rec['Max'])
rec['Recency'] = (curr_date - rec['Max']).astype('timedelta64[D]')
rec['Time'] = (rec['Max'] - rec['Min']).astype('timedelta64[D]')
rec = rec[['card_id', 'Recency', 'Time']]
rec

,card_id,Recency,Time
0,C_ID_00007093c1,295.0,6.0
1,C_ID_0001238066,274.0,60.0
2,C_ID_0001506ef0,313.0,5.0
3,C_ID_0001793786,394.0,46.0
4,C_ID_000183fdda,274.0,59.0
5,C_ID_0002709b5a,274.0,58.0
6,C_ID_000298032a,306.0,0.0
7,C_ID_0002ba3c2e,413.0,40.0
8,C_ID_0002c7c2c1,311.0,44.0
9,C_ID_00032df08f,365.0,23.0


In [382]:
freq = hist_df.groupby('card_id').size().reset_index()
freq.columns = ['card_id', 'Frequency']

mon = hist_df.groupby('card_id')['purchase_amount'].sum().reset_index()
mon.columns = ['card_id', 'Monitary']

final = pd.merge(freq, mon, how='left', on='card_id')
final = pd.merge(final, rec, how='left', on='card_id')
final.head()

,card_id,Frequency,Monitary,Recency,Time
0,C_ID_00007093c1,149,-76.845041,295.0,6.0
1,C_ID_0001238066,123,-72.447201,274.0,60.0
2,C_ID_0001506ef0,66,-34.601879,313.0,5.0
3,C_ID_0001793786,216,-36.786013,394.0,46.0
4,C_ID_000183fdda,144,-68.837938,274.0,59.0


In [384]:
final['historic_CLV'] = final['Frequency'] * final['Monitary']
final['AOV'] = final['Monitary']/final['Frequency']
final['Predictive_CLV'] = final['Time']*final['AOV']*final['Monitary']*final['Recency'] 
final.head()

,card_id,Frequency,Monitary,Recency,Time,historic_CLV,AOV,Predictive_CLV
0,C_ID_00007093c1,149,-76.845041,295.0,6.0,-11449.911042,-0.515739,70148.548021
1,C_ID_0001238066,123,-72.447201,274.0,60.0,-8911.005737,-0.589002,701519.787286
2,C_ID_0001506ef0,66,-34.601879,313.0,5.0,-2283.724033,-0.524271,28390.286801
3,C_ID_0001793786,216,-36.786013,394.0,46.0,-7945.778765,-0.170306,113544.404678
4,C_ID_000183fdda,144,-68.837938,274.0,59.0,-9912.663098,-0.478041,531980.594261


In [388]:
new_additional_features = final

In [379]:
hist_additional_features = final

In [390]:
hist_additional_features.add_prefix('hist_')

,hist_card_id,hist_Frequency,hist_Monitary,hist_Recency,hist_Time,hist_historic_CLV,hist_AOV,hist_Predictive_CLV
0,C_ID_00007093c1,149,-76.845041,337.0,377.0,-11449.911042,-0.515739,5.035199e+06
1,C_ID_0001238066,123,-72.447201,336.0,151.0,-8911.005737,-0.589002,2.164982e+06
2,C_ID_0001506ef0,66,-34.601879,346.0,398.0,-2283.724033,-0.524271,2.498128e+06
3,C_ID_0001793786,216,-36.786013,455.0,283.0,-7945.778765,-0.170306,8.066953e+05
4,C_ID_000183fdda,144,-68.837938,338.0,202.0,-9912.663098,-0.478041,2.246784e+06
5,C_ID_00024e244b,70,-41.552753,357.0,399.0,-2908.692707,-0.593611,3.513522e+06
6,C_ID_0002709b5a,73,-47.659542,344.0,402.0,-3479.146594,-0.652870,4.302901e+06
7,C_ID_00027503e2,42,-31.123047,375.0,237.0,-1307.167954,-0.741025,2.049720e+06
8,C_ID_000298032a,30,-15.992416,370.0,345.0,-479.772478,-0.533081,1.088248e+06
9,C_ID_0002ba3c2e,70,-44.342996,457.0,295.0,-3104.009712,-0.633471,3.786956e+06


In [393]:
hist_additional_features.iloc[:,1:] = hist_additional_features.iloc[:,1:].add_prefix('hist_')

In [401]:
['authorized_flag', 'category_1', 'category_2', 'category_3']

['authorized_flag', 'category_1', 'category_2', 'category_3']

In [403]:
hist_df['subsector_id'].unique()

array([37, 16, 34, 41, 29,  7, 33,  2, 36, 25, 15, 19, 10, 30, 18,  8, 27,
        1, 39, 32, 22, 20, 21,  9, 17, 38, 12, 26, 31, 13, 35,  5, 23,  3,
        4, 14, 24, 40, 11, -1, 28])

In [398]:
hist_df

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37
5,Y,C_ID_4e6213e9bc,333,N,0,A,80,M_ID_50af771f8d,0,-0.734887,2018-02-24 08:45:05,1.0,9,37
6,Y,C_ID_4e6213e9bc,88,N,0,A,278,M_ID_5e8220e564,-11,-0.716855,2017-03-21 00:10:51,1.0,16,37
7,Y,C_ID_4e6213e9bc,3,N,0,A,80,M_ID_9d41786a50,-3,-0.657049,2017-11-18 20:05:55,1.0,16,37
8,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-8,-0.737967,2017-06-01 22:02:56,1.0,16,37
9,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_74ba14b5fc,-11,-0.715352,2017-03-16 15:41:22,1.0,16,37


In [400]:
new_merchant_df

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29
5,Y,C_ID_ef55cf8d4b,231,N,1,B,367,M_ID_8874615e00,2,-0.633007,2018-04-02 12:53:28,1.0,9,16
6,Y,C_ID_ef55cf8d4b,69,N,1,B,333,M_ID_6d061b5ddc,1,5.263697,2018-03-28 19:50:19,1.0,9,10
7,Y,C_ID_ef55cf8d4b,231,N,1,B,307,M_ID_df1e022f41,2,-0.553787,2018-04-05 08:06:52,1.0,9,19
8,Y,C_ID_ef55cf8d4b,69,N,1,B,278,M_ID_d15eae0468,2,-0.596643,2018-04-07 18:37:40,1.0,9,37
9,Y,C_ID_ef55cf8d4b,69,N,1,B,437,M_ID_5f9bffd028,1,-0.607191,2018-03-17 18:10:41,1.0,9,15


In [404]:
merchants

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,city_id,state_id,numerical_ratio,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_sales_range_nan,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,most_recent_purchases_range_nan,active_months_lag3_1.0,active_months_lag3_2.0,active_months_lag3_3.0,active_months_lag3_nan,active_months_lag6_1.0,active_months_lag6_2.0,active_months_lag6_3.0,active_months_lag6_4.0,active_months_lag6_5.0,active_months_lag6_6.0,active_months_lag6_nan,active_months_lag12_1.0,active_months_lag12_2.0,active_months_lag12_3.0,active_months_lag12_4.0,active_months_lag12_5.0,active_months_lag12_6.0,active_months_lag12_7.0,active_months_lag12_8.0,active_months_lag12_9.0,active_months_lag12_10.0,active_months_lag12_11.0,active_months_lag12_12.0,active_months_lag12_nan,category_1_N,category_1_Y,category_1_nan,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_4_N,category_4_Y,category_4_nan,has_last_month_transaction,has_last3_months_revenue
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,-0.40,9.666667,-2.25,18.666667,-2.32,13.916667,242,9,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,-0.72,1.750000,-0.74,1.291667,-0.57,1.687500,22,16,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,-82.13,260.000000,-82.13,260.000000,-82.13,260.000000,-1,5,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,0.00,1.666667,0.00,4.666667,0.00,3.833333,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,0.00,0.500000,0.00,0.361111,0.00,0.347222,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
5,M_ID_a0915f62b5,20201,87,27,-0.057471,-0.057471,0.00,1.000000,0.00,3.666667,0.00,3.833333,160,21,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0
6,M_ID_bfd41933db,33861,792,9,-0.057471,-0.057471,0.00,2.833333,0.00,4.833333,0.00,6.333333,60,16,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
7,M_ID_d8ff08219e,16430,529,20,-0.057471,-0.057471,0.00,1.000000,0.00,1.666667,0.00,1.500000,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
8,M_ID_c5b389236d,37179,813,29,-0.057471,-0.057471,0.00,115.000000,0.00,189.916667,0.00,197.000000,248,15,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
9,M_ID_d2162ed113,112122,81,29,-0.057471,-0.057471,0.00,1.000000,0.00,1.000000,0.00,1.000000,-1,-1,1.000000,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
